In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __init__ import *

In [ ]:
assert SETTING["signal_type"] == "raw"

In [ ]:
feature_x_sample_processed = pd.read_csv(
    PATH["feature_x_sample.processed.tsv"], sep="\t", index_col=0
)

feature_x_sample_processed.columns.name = SETTING["sample_alias"]

In [ ]:
layout = {"yaxis": {"title": {"text": "Raw Signal Standard Deviation"}}}

In [ ]:
standard_deviation = SETTING["select_feature_standard_deviation"]

if standard_deviation is not None:

    features = kraft.select_series_indices(
        feature_x_sample_processed.std(axis=1),
        ">",
        standard_deviation=standard_deviation,
        layout=layout,
    )

else:

    features = feature_x_sample_processed.index

In [ ]:
standard_deviation = SETTING["select_sample_standard_deviation"]

if standard_deviation is not None:

    samples = kraft.select_series_indices(
        feature_x_sample_processed.std(),
        ">",
        standard_deviation=standard_deviation,
        layout=layout,
    )

else:

    samples = feature_x_sample_processed.columns

In [ ]:
raw_signal = feature_x_sample_processed.loc[features, samples]

In [ ]:
if SETTING["signal_normalization_method"] is not None:

    raw_signal = kraft.normalize_dataframe(
        raw_signal,
        SETTING["signal_normalization_axis"],
        SETTING["signal_normalization_method"],
    )

In [ ]:
raw_signal.to_csv(PATH["feature_x_sample.signal.tsv"], sep="\t")

kraft.summarize_feature_x_sample(
    raw_signal,
    feature_x_sample_alias=SETTING["feature_x_sample_alias"],
    feature_x_sample_value_name="Signal",
)